In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import logging
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

logging.basicConfig(filename='webscrape.log' , format='%(asctime)s | %(levelname)s: %(message)s', level=logging.NOTSET)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kisho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Misc func.

def print_df(df):
    for i in df:
        print(i)

In [3]:
def get_loc(result):
    try:
        return entry.find('div', {'class': 'companyLocation'}).text
    except Exception as e:
        return 'NA'


def get_comp(result):
    try:
        return result.find('span', {'class': 'companyName'}).text
    except:
        return 'NA'


def get_job(result):
    try:
        return result.find('h2', {'class': 'jobTitle'}).text
    except:
        return 'NA'


def get_sal(result):
    try:
        return result.find('div', {'class': 'salaryOnly'}).text
    except:
        return 'NA'


def get_desc(result):
    try:
        return result.find('div', {'class': 'job-snippet'}).text
    except:
        return 'NA'
def get_rating(result):
    try:
        return result.find('span', {'class': 'ratingNumber'}).text
    except:
        return 'NA'

In [4]:
from time import sleep
import pandas as pd
import numpy as np

In [ ]:
job_titles = [
    'Data+Scientist', 'full+stack+developer', 'big+data+engineer',
    'software+architect', 'blockchain', 'devops+engineer', 'cloud+architect',
    'artificial+intelligence', 'product+manager'
]

len(job_titles)

In [ ]:
results = []

In [ ]:
# max_results_per_job = 2000
# try:
#     for job in job_titles:
#         print("-------------- Scrapping for : " + job  + " Started --------------------------")
#         logging.info("-------------- Scrapping for : " + job  + " Started --------------------------")
#         for start in range(0, max_results_per_job, 100):
#             url = "https://in.indeed.com/jobs?q=" + job + "&start=" + str(start) + '&limit=100'
#             html = requests.get(url) 
#             soup = BeautifulSoup(html.text, 'html.parser')
#             for result in soup.find_all('div', {'class':'job_seen_beacon'}):
                
#                 results.append(result)
#             sleep(5)
#         logging.info("Gathered " + str(len(results)) + " jobs so far ...")
#         print("Gathered " + str(len(results)) + " jobs so far ...")
#         print("-------------- Scrapping for : " + job  + " Completed --------------------------")
#         logging.info("-------------- Scrapping for : " + job  + " Completed -------------------------")
#     print("Scraping done !!! Check logs for more info")
# except Exception as e:
#     logging.error(e)
#     print("Something went wrong :(. Please check logs")

In [ ]:
jobs0 = pd.DataFrame(columns=['location', 'title', 'company', 'salary', 'summary', 'rating'])

In [ ]:
for entry in results:
    location = get_loc(entry)
    title = get_job(entry)
    company = get_comp(entry)
    salary = get_sal(entry)
    desc = get_desc(entry)
    rating = get_rating(entry)
    jobs0.loc[len(jobs0)] = [location, title, company, salary, desc, rating]

In [ ]:
jobs0.drop_duplicates(inplace = True, keep='first')
jobs0
jobs0.to_csv(r'./Dataset/job_raw.csv')
# jobs0.describe()

In [5]:
df_jobs = pd.read_csv(r'./Dataset/job_raw.csv')

In [6]:
len(df_jobs)

5418

In [11]:
df_jobs[df_jobs.duplicated()]
df_jobs.drop(columns=['Unnamed: 0'], inplace=True)

In [12]:
display(df_jobs.describe())
print(df_jobs.isna().sum())

,rating
count,3026.000000
mean,3.833179
std,0.347396
min,1.000000
25%,3.700000
50%,3.900000
75%,4.000000
max,5.000000


location       0
title          0
company        0
salary      4102
summary        0
rating      2392
dtype: int64


In [13]:
df_jobs.replace(to_replace='NA', value=np.nan, inplace=True)

In [14]:
df = df_jobs

In [16]:
print('Unique Locations  \t:', len(df.location.unique()))
print('Unique Title  \t\t:', len(df.title.unique()))
print('Unique Company  \t:', len(df.company.unique()))
print('Unique Salary \t\t:', len(df.salary.unique()))
print('Unique Rating \t\t:', len(df.rating.unique()))
# Unique Locations  	: 385
# Unique Title  		: 3409
# Unique Company  	: 2426
# Unique Salary 		: 605
# Unique Rating 		: 31

Unique Locations  	: 385
Unique Title  		: 3409
Unique Company  	: 2426
Unique Salary 		: 605
Unique Rating 		: 31


In [ ]:
"""
    * Noise found
         1. 'Remote in '
         2. '+1 location'
         3. 'Hybrid remote in '
         4. '+7 location'
         5. '+2 location'
         6. 'Temporarily Remote in '

"""



In [19]:
stop_words = stopwords.words('english') +  ['in', 'remote', 'Remote', 'location', 'locations','Temporarily', 'Hybrid', 'India']
df['clean_location'] = df['location'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


def clean_loc(location):
    remove_list = ['+1', '+2', '+3','+4','+5','+6','+7','+8', '+9']
    for i in remove_list:
        if i in location:
            return location.replace(i, '')
    return location

df.clean_location = df.clean_location.apply(clean_loc)
df.clean_location = df.clean_location.str.strip()

In [29]:
df

,location,title,company,salary,summary,rating,clean_location,clean_title
0,"Bengaluru, Karnataka",Data Scientist,Applied Materials Inc.,"₹5,26,846 - ₹17,85,107 a year","\nAcquire, cleanse, organize and mine massive ...",3.9,"Bengaluru, Karnataka",Data Scientist
1,"Bengaluru, Karnataka",Senior Data Scientist,CORESTRAT LABS PRIVATE LIMITED,"₹30,00,000 a year",\nCollaborate to enhance data infrastructure a...,NaN,"Bengaluru, Karnataka",Senior Data Scientist
2,"Noida, Uttar Pradesh",Tech Mahindra hiring for Data Scientist - Noida,Tech Mahindra,"₹4,00,000 - ₹16,00,000 a year","\nData scientist: 3 years (Required).\nPython,...",NaN,"Noida, Uttar Pradesh",Tech Mahindra hiring for Data Scientist - Noida
3,"Bengaluru, Karnataka",newData Scientist,Happay,"₹8,00,000 - ₹12,00,000 a year","\n1-2 Years as a data scientist, machine learn...",NaN,"Bengaluru, Karnataka",newData Scientist
4,"Chennai, Tamil Nadu+1 location",newAssociate Data Scientist,Shell,NaN,"\nGathers data, analyses and reports findings....",3.9,"Chennai, Tamil Nadu",newAssociate Data Scientist
...,...,...,...,...,...,...,...,...
5413,"Vikhroli, Mumbai, Maharashtra",Team Lead Designer,Accelya,NaN,\nExperienced in agile ways of working and abl...,3.6,"Vikhroli, Mumbai, Maharashtra",Team Lead Designer
5414,"Hyderabad, Telangana",newSenior Full Stack Web Developer,Avidbots,NaN,\nDesign the technical specification to match ...,NaN,"Hyderabad, Telangana",newSenior Full Stack Web Developer
5415,"Bengaluru, Karnataka",Inside Sales Account Management,Dell Technologies,NaN,\nMay include some enterprise products and ser...,4.0,"Bengaluru, Karnataka",Inside Sales Account Management
5416,"Bengaluru, Karnataka",Technical Staff Software Engineer,Dell Technologies,NaN,"\nResearch, evaluate and contribute to project...",4.0,"Bengaluru, Karnataka",Technical Staff Software Engineer


In [21]:
print('Uncleaned Locations  \t:', len(df.location.unique()))
print('Cleaned Locations  \t:', len(df.clean_location.unique()))

Uncleaned Locations  	: 385
Cleaned Locations  	: 207


In [31]:
print_df(df.title.unique())

Data Scientist
Senior Data Scientist
Tech Mahindra hiring for Data Scientist - Noida
newData Scientist
newAssociate Data Scientist
Data Scientist - SQL Queries
newClinical Data Scientist
Senior Data scientist
newClinical Data Management Scientist
newJunior Data Scientist
newData & Applied Scientist
DATA SCIENTIST DEMAND FORECASTING
newManager, Senior Clinical Data Scientist, Clinical Data Scien...
Data Scientist (Analyst)
newData Science Analyst
Data Scientist - Predictive Analysis, Time Series Forecastin...
Data Scientist: Advanced Analytics
newData Scientist / Analyst
newData Scientist: Artificial Intelligence
Data Scientist (Pricing Analytics)
newData Scientist - Life and Health
newBattery Analytics Data Scientist
Data & Analytics Analyst
Data Scientist (Operation Research)
Manager - Financial Insights & Analytics (Data Scientist)
newData Scientist - OrgSolutions
Data Scientist- Clinical Trial
newDeep Learning Specialist (Data Scientist)
Data Scientist-Geospatial Analytics
Big Data 

newAssistant Manager_Senior Engineer_ UI UX_Pune
Technical Expert, Cloud Architect
Cloud | Salesforce Architect
Workplace Collaboration Services Architect- Consulting Archi...
CLOUD APPLICATION/PRODUCT ARCHITECT
Azure Cloud Architect
Lead Cloud Architect
Pivotal Cloud Foundry-Architect
Cloud Architect (GCP)
newCisco HyperFlex Architecture Infrastructure Architect
SFCC Architect
Chief Technology Architect
Public Cloud Director
GCP Architect
newPrincipal Architect
Cloud-PaaS-Azure-Architect
newCloud and Enterprise Systems Architect
newAnalyst, Information Security
Senior Manager - Siebel Architect
Architect Kafka Cloud - Multiple Locations
Bid Architect
Azure Cloud Solution Architect
Solutions Architect-AWS.
QlikSense Architect
Application Architect: Azure Cloud Modernization
Sr. Cloud Architect
Cloud Security - GCP - Application Architect
AWS ITOps Engineers
AEM Architect
AWS Cloud Architect
newAssociate Director,Solutions Architect
Software Sales Specialist
AEM Sr Architect - WCM - Dig

In [30]:
stop_words = stopwords.words('english') +  ['Scien...', 'Forecastin...', '-', 'location', 'locations','Temporarily', 'Hybrid', 'India']
df['clean_title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [28]:
def clean_titl(title):
    remove_list = ['new', '-', 'Clinical']
    for i in remove_list:
        if i in title:
            return title.replace(i, '')
    return title

df.clean_title = df.clean_title.apply(clean_titl)
df.clean_title = df.title.str.strip()

In [ ]:
test = 'newManager, Senior - Clinical Data Scientist, Clinical Data Scien...'
if 'new' in test:
    print(True)

In [ ]:
print_df(df.clean_title)

In [ ]:
df['clean_title']

In [34]:
df_comp_rat = df_jobs[['company', 'rating']]

In [40]:
df_comp_rat.drop_duplicates(keep='first')

,company,rating
0,Applied Materials Inc.,3.9
1,CORESTRAT LABS PRIVATE LIMITED,NaN
2,Tech Mahindra,NaN
3,Happay,NaN
4,Shell,3.9
...,...,...
5377,ANRA Technologies Pvt. Ltd.,NaN
5385,CIMB THAI,NaN
5399,MineralTree,NaN
5403,LeadVenture India,3.6
